In [1]:
import conference_scheduler
from conference_scheduler.resources import Slot, Event
import pulp
import itertools

In [2]:
assert conference_scheduler.__version__ == "1.1.0"

In [3]:
rooms = ["Assembly room", "Room D", "Ferrier Hall", "Room C"]
days = ['16-Sep-2016', '17-Sep-2016', '18-Sep-2016']
times_and_durations = [('10:15', 30), ('11:15', 45), ('12:00', 30), ('12:30', 30), ('14:30', 30), 
                       ('15:00', 30), ('15:30', 30), ('16:30', 30), ('17:00', 30)]
day_period = {('10:15', 30) : "Morning A",
              ('11:15', 45) : "Morning B", 
              ('12:00', 30) : "Morning B",
              ('12:30', 30) : "Morning B",
              ('14:30', 30) : "Afternoon", 
              ('15:00', 30) : "Afternoon", 
              ('15:30', 30) : "Afternoon",
              ('16:30', 30) : "Evening", 
              ('17:00', 30) : "Evening"}

room_capacity = {"Assembly room": 500,
                 "Room D": 100,
                 "Ferrier Hall": 200,
                 "Room C": 80}

In [4]:
talk_slots = []
for room, day, time_and_duration in itertools.product(rooms, days, times_and_durations):
    if (room, day) not in [("Room C", '16-Sep-2016'), 
                           ("Room C", '18-Sep-2016')]: # Rooms, Days used for workshops
        time, duration = time_and_duration
        session = f"Talks: {day} {day_period[time_and_duration]}"
        starts_at = f"{day} {time}"
        capacity = room_capacity[room]
        talk_slots.append(Slot(venue=room, starts_at=starts_at, duration=duration, session=session, capacity=capacity))

In [5]:
len(talk_slots)

90

In [6]:
rooms = ["Room C", "Room A"]
days = ['18-Sep-2016']
times_and_durations = [('10:15', 45), ('11:15', 105), ('14:30', 90), ('16:30', 60)]

In [7]:
workshop_slots = []

for room, day, time_and_duration in itertools.product(rooms, days, times_and_durations):
        time, duration = time_and_duration
        session = f"Workshop: {day} {time}"
        starts_at = f"{day} {time}"
        capacity = 80
        workshop_slots.append(Slot(venue=room, starts_at=starts_at, duration=duration, session=session, capacity=capacity))

In [8]:
len(workshop_slots)

8

In [9]:
plenary_slots = [Slot(venue="Assembly Room", starts_at='16-Sep-2016 09:10', duration=50, session="Plenary: 16-Sep-2016", capacity=500),
                 Slot(venue="Assembly Room", starts_at='17-Sep-2016 09:10', duration=50, session="Plenary: 17-Sep-2016", capacity=500),
                 Slot(venue="Assembly Room", starts_at='18-Sep-2016 09:10', duration=50, session="Plenary: 18-Sep-2016", capacity=500)]

In [10]:
slots = talk_slots + plenary_slots + workshop_slots

In [11]:
import csv
with open("../data/pycon_2016_talks.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # Throw away the header
    raw_talks = list(reader)

In [12]:
talks = []
for row in raw_talks:
    name = row[0]
    duration = int(row[1])
    tags = [row[2]] if row[2] != '' else []
    talks.append(Event(name=name, duration=duration, tags=tags, unavailability=plenary_slots[:], demand=None))

In [13]:
len(talks)

69

In [14]:
education_talks = [talk for talk in talks if "Education" in talk.tags]
slots_not_on_last_day = [slot for slot in talk_slots if ('16-Sep-2016' in slot.starts_at) or ('17-Sep-2016' in slot.starts_at)]
for talk in education_talks:
    talk.unavailability.extend(education_talks[:])
    talk.unavailability.extend(slots_not_on_last_day)

In [15]:
plenary_talks = [Event(name="Python and the Glories of the UNIX Tradition", 
                 duration=50, tags=[], unavailability=talk_slots[:] + workshop_slots[:], demand=None),
                 Event(name="Folklore and fantasy in the information age", 
                 duration=50, tags=[], unavailability=talk_slots[:] + workshop_slots[:], demand=None),
                 Event(name="An Arabish lesson: Introducing Django to the foreign world", 
                 duration=50, tags=[], unavailability=talk_slots[:] + workshop_slots[:], demand=None)]

In [16]:
workshops = [Event(name="Open Data projects with Python", duration=45, tags=[], 
                   unavailability=talk_slots[:] + plenary_slots[:], 
                   demand=None),
             Event(name="Dive Into Object-oriented Python", duration=45, tags=[], 
                   unavailability=talk_slots[:] + plenary_slots[:], 
                   demand=None),
             Event(name="An introduction to deep learning", duration=105, tags=[], 
                   unavailability=talk_slots[:] + plenary_slots[:], 
                   demand=None),
             Event(name="Dive Into Object-oriented Python (cont.)", duration=105, tags=[], 
                   unavailability=talk_slots[:] + plenary_slots[:], 
                   demand=None),
             Event(name="Python for Scientists (feat. Software Carpentry)", duration=90, tags=[], 
                   unavailability=talk_slots[:] + plenary_slots[:], 
                   demand=None),
             Event(name="Data Wrangling with Python", duration=90, tags=[], 
                   unavailability=talk_slots[:] + plenary_slots[:], 
                   demand=None),
             Event(name="Users are not the only people", duration=60, tags=[], 
                   unavailability=talk_slots[:] + plenary_slots[:], 
                   demand=None)]

In [17]:
events = talks + plenary_talks + workshops

In [18]:
%%time
from conference_scheduler import scheduler
schedule = scheduler.schedule(events=events, slots=slots, solver=pulp.GLPK())

CPU times: user 12.9 s, sys: 152 ms, total: 13.1 s
Wall time: 24.2 s


In [19]:
schedule = sorted(schedule, key=lambda item: item.slot.starts_at)

In [20]:
for item in schedule:
    if item.event in workshops:
        print(f"{item.event.name} at {item.slot.starts_at} in {item.slot.venue}")

Open Data projects with Python at 18-Sep-2016 10:15 in Room C
An introduction to deep learning at 18-Sep-2016 11:15 in Room C
Dive Into Object-oriented Python (cont.) at 18-Sep-2016 11:15 in Room A
Python for Scientists (feat. Software Carpentry) at 18-Sep-2016 14:30 in Room C
Data Wrangling with Python at 18-Sep-2016 14:30 in Room A
Dive Into Object-oriented Python at 18-Sep-2016 16:30 in Room C
Users are not the only people at 18-Sep-2016 16:30 in Room A


In [21]:
for item in schedule:
    if item.event in plenary_talks:
        print(f"{item.event.name} at {item.slot.starts_at} in {item.slot.venue}")

Python and the Glories of the UNIX Tradition at 16-Sep-2016 09:10 in Assembly Room
Folklore and fantasy in the information age at 17-Sep-2016 09:10 in Assembly Room
An Arabish lesson: Introducing Django to the foreign world at 18-Sep-2016 09:10 in Assembly Room


In [22]:
for item in schedule:
    if item.event in talks:
        print(f"{item.event.name} at {item.slot.starts_at} in {item.slot.venue}")

Transforming the government’s Digital Marketplace from portal to platform at 16-Sep-2016 10:15 in Assembly room
The state of PyPy at 16-Sep-2016 11:15 in Room D
An Introduction to Deep Learning with TensorFlow at 16-Sep-2016 11:15 in Ferrier Hall
Simulating a CPU with Python or: surprising programs you might have thought were better written in C at 16-Sep-2016 11:15 in Assembly room
Distributed systems from scratch: lessons learned the hard way! at 16-Sep-2016 12:00 in Assembly room
Taking control of your Bluetooth devices at 16-Sep-2016 12:00 in Room D
Assessing performance of Support Vector Machine kernels to detect interactions in genotyped data at 16-Sep-2016 12:00 in Ferrier Hall
An Introduction to web scraping using Python at 16-Sep-2016 12:30 in Assembly room
A data processing toolbox for agile scientific research at 16-Sep-2016 12:30 in Room D
Lessons learned from organizing SciPy Latin America 2016 at 16-Sep-2016 12:30 in Ferrier Hall
Timezones: A tale of (more than) two citie

In [23]:
for item in schedule:
    if item.event in talks and "Education" in item.event.tags:
        print(f"{item.event.name} at {item.slot.starts_at} in {item.slot.venue}")

Developing CS education - how can you help at 18-Sep-2016 11:15 in Assembly room
Why do kids need to code and how can we help? at 18-Sep-2016 12:00 in Assembly room
Children's Day Show and Tell at 18-Sep-2016 12:30 in Room D
